In [2]:
import pandas as pd
import altair as alt
import numpy as np
from pathlib import Path
from helpers import read, write

import bottleneck as bn
from LPA import Corpus, sockpuppet_distance
from math import floor
from scipy.spatial.distance import cdist

from visualize import sockpuppet_matrix, timeline

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [3]:
for i in (0, 1):
    freq = pd.read_csv(f"data/us_elections/np_freq/{i}.csv")
    pop = pd.read_csv("data/us_elections/population.csv")
    pop = (
        pop.filter(freq["document"].drop_duplicates().astype(str).tolist() + ["state"])
        .melt(var_name="document", id_vars=["state"])
        .rename(columns={"state": "element"})
    )
    pop["value"] = pop["value"].str.replace(",", "").astype(int)
    pop["document"] = pop["document"].astype(int)
    display(pop)
    df = pd.merge(freq, pop, how="left", on=["document", "element"])
    df["frequency_in_document"] /= df["value"]
    display(df)
    df = df.drop(columns=["value", "party_simplified"])

    # corpus = Corpus(df)
    # dvr = corpus.create_dvr(equally_weighted=True)
    # sigs = corpus.create_signatures(epsilon=1 / (len(dvr) * 2))
    # display(timeline(
    #     df,
    #     x="document",
    #     y="frequency_in_document",
    #     subcorpus="us_elections",
    #     stack="center",
    #     order=dvr["element"].tolist(),
    #     name=f"{i}",
    # ))
    # display(
    #     sockpuppet_matrix(sockpuppet_distance(corpus, corpus))
    #     .properties(title=(("Democrat" if i == 0 else "Republican")))
    #     .configure_axis(title=None)
    # )
    # sockpuppet_matrix(sockpuppet_distance(corpus, corpus)).properties(
    #     title=(("Democrat" if i == 0 else "Republican"))
    # ).configure_axis(title=None).save(
    #     f"results/us_elections/{i}_normalized_population.html"
    # )

,element,document,value
0,ALABAMA,1976,3737204
1,ALASKA,1976,393115
2,ARIZONA,1976,2347976
3,ARKANSAS,1976,2168688
4,CALIFORNIA,1976,21934505
...,...,...,...
607,VIRGINIA,2020,8632044
608,WASHINGTON,2020,7718785
609,WEST VIRGINIA,2020,1789798
610,WISCONSIN,2020,5892323


,document,frequency_in_document,party_simplified,element,value
0,1976,0.176381,DEMOCRAT,ALABAMA,3737204
1,1976,0.112074,DEMOCRAT,ALASKA,393115
2,1976,0.125897,DEMOCRAT,ARIZONA,2347976
3,1976,0.229910,DEMOCRAT,ARKANSAS,2168688
4,1976,0.170612,DEMOCRAT,CALIFORNIA,21934505
...,...,...,...,...,...
610,2020,0.279606,DEMOCRAT,VIRGINIA,8632044
611,2020,0.306993,DEMOCRAT,WASHINGTON,7718785
612,2020,0.131850,DEMOCRAT,WEST VIRGINIA,1789798
613,2020,0.276778,DEMOCRAT,WISCONSIN,5892323


,element,document,value
0,ALABAMA,1976,3737204
1,ALASKA,1976,393115
2,ARIZONA,1976,2347976
3,ARKANSAS,1976,2168688
4,CALIFORNIA,1976,21934505
...,...,...,...
607,VIRGINIA,2020,8632044
608,WASHINGTON,2020,7718785
609,WEST VIRGINIA,2020,1789798
610,WISCONSIN,2020,5892323


,document,frequency_in_document,party_simplified,element,value
0,1976,0.134879,REPUBLICAN,ALABAMA,3737204
1,1976,0.182021,REPUBLICAN,ALASKA,393115
2,1976,0.178299,REPUBLICAN,ARIZONA,2347976
3,1976,0.123532,REPUBLICAN,ARKANSAS,2168688
4,1976,0.176993,REPUBLICAN,CALIFORNIA,21934505
...,...,...,...,...,...
608,2020,0.227342,REPUBLICAN,VIRGINIA,8632044
609,2020,0.205298,REPUBLICAN,WASHINGTON,7718785
610,2020,0.304717,REPUBLICAN,WEST VIRGINIA,1789798
611,2020,0.273268,REPUBLICAN,WISCONSIN,5892323


In [67]:
for i in (0, 1):
    freq = pd.read_csv(f"data/us_elections/np_freq/{i}.csv")
    x = pd.merge(
        freq,
        freq.drop(columns=["party_simplified"])
        .groupby("document")
        .sum(numeric_only=True),
        left_on="document",
        right_index=True,
    )
    x["frequency_in_document"] = (
        x["frequency_in_document_x"] / x["frequency_in_document_y"]
    )
    x = x.drop(
        columns=[
            "frequency_in_document_x",
            "frequency_in_document_y",
            "party_simplified",
        ]
    )
    cc = x.pivot_table(
        index="document", columns="element", values="frequency_in_document"
    ).to_numpy()
    dm = pd.DataFrame(
        cdist(cc, cc, metric="cityblock"),
        index=corpus.document_cat.categories,
        columns=corpus.document_cat.categories,
    )
    dm /= dm.max().max()
    c1n = getattr(corpus, "name", "Corpus 2")
    c2n = getattr(corpus, "name", "Corpus 1")
    if c1n == c2n:
        c2n = c1n + " "
    dm = dm.rename_axis(index=c1n).melt(ignore_index=False, var_name=c2n).reset_index()
    # display()
    display(
        sockpuppet_matrix(dm)
        .properties(title=(("Democrat" if i == 0 else "Republican")))
        .configure_axis(title=None)
    )
    sockpuppet_matrix(dm).properties(
        title=(("Democrat" if i == 0 else "Republican"))
    ).configure_axis(title=None).save(
        f"results/us_elections/{i}_raw_data_sockpuppets.html"
    )

/home/alexzabbey/.virtualenvs/tlpa/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

/home/alexzabbey/.virtualenvs/tlpa/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [ ]:
# df = pd.read_csv("data/us_elections/1976-2020-president.csv")[["year", "candidatevotes", "party_simplified", "state"]]
# df = df[df["party_simplified"].isin(["DEMOCRAT", "REPUBLICAN"])].rename(columns={"state": "element", "year": "document", "candidatevotes": "frequency_in_document"}).reset_index(drop=True)
# df[df["party_simplified"] == "DEMOCRAT"].reset_index(drop=True).to_csv("data/us_elections/np_freq/0.csv", index=False)
# cs = []
# for party in ["DEMOCRAT", "DEMOCRAT"]:
#     corpus = Corpus(df[df["party_simplified"] == party].reset_index(drop=True))
#     dvr = corpus.create_dvr()
#     sigs = corpus.create_signatures(epsilon=1/(len(dvr) * 2))
#     cs.append(corpus)
# spd = sockpuppet_distance(*cs)
# sockpuppet_matrix(spd, "DEMOCRAT", "DEMOCRAT 1") #.save("dem_vs_dem.html")

In [ ]:
# corpus = Corpus(df[df["party_simplified"] == "DEMOCRAT"].reset_index(drop=True))
# dvr = corpus.create_dvr(equally_weighted=True)
# sigs = corpus.create_signatures(epsilon=1/(len(dvr) * 2))

In [29]:
# for i, s in enumerate(sigs[0]):
#     s.to_csv(f"results/us_elections/signatures/{1976 + i * 4}.csv")
# dvr.to_csv("results/us_elections/dvr.csv", index=False)
# # sigs[0][8].to_csv("2008.csv")
x = np.array([[1, 2, 3], [4, 5, 6]])
x = (x.T / x.sum(axis=1)).T
x
# average_weight = bn.nanmean(x, axis=0)
# average_weight
# average_weight / average_weight.sum()

array([[0.16666667, 0.33333333, 0.5       ],
       [0.26666667, 0.33333333, 0.4       ]])

In [91]:
ddf = (
    pd.read_csv("data/death_cause/annual-number-of-deaths-by-cause.csv")
    .melt(
        id_vars=["Entity", "Code", "Year"],
        var_name="element",
        value_name="frequency_in_document",
    )
    .fillna(0)
    .rename(columns={"Year": "document"})
)
ddf["frequency_in_document"] = ddf["frequency_in_document"].astype(int)
dddf = ddf.groupby(["document", "element"], as_index=False).sum(numeric_only=True)
popp = pd.read_csv("data/death_cause/population.csv").rename(
    columns={"Year": "document"}
)
dff = pd.merge(dddf, popp, how="left")
dff["frequency_in_document"] /= dff["Population"]
dff = dff.drop(columns=["Population"])

In [102]:
corpus = Corpus(dff)
dvr = corpus.create_dvr(equally_weighted=True)
dvr.to_csv("results/death_cause/dvr.csv", index=False)
epsilon = 1 / (len(dvr) * 2)
sigs = corpus.create_signatures(epsilon=epsilon, prevelent=0.75)
for sig in sigs[0]:
    name = sig.name
    sig = sig.rename("KL").reset_index()
    # wordlist[name] = sig[sig["index"] == word].loc[:, "KL"].iloc[0]
    write(
        Path("results/death_cause"),
        sig,
        f"sigs/sigs_with_prevelence_0.75_{name}_epsilon_{epsilon}",
        color=True,
    )
display(
    sockpuppet_matrix(sockpuppet_distance(corpus, corpus))
    .properties(title=("Cause of death"))
    .configure_axis(title=None)
)

wrote sigs/sigs_with_prevelence_0.75_1990_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1991_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1992_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1993_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1994_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1995_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1996_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1997_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1998_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_1999_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_2000_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_2001_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_2002_epsilon_0.016129032258064516
wrote sigs/sigs_with_prevelence_0.75_2003_epsilon_0.016129032258064516
wrote 

/home/alexzabbey/.virtualenvs/tlpa/lib/python3.10/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [107]:
timeline(
    dff,
    x="document",
    y="frequency_in_document",
    subcorpus="death_cause",
    stack="center",
    order=dvr["element"].tolist(),
)

alt.Chart(...)